# Állomások lokációi és a legközelebbi meteorológia mérőállomás

In [5]:
import os

import pandas as pd

In [6]:
from config import data_root,weather_folder,gtfs_folder,gtfs_stops_file,passanger_info_folder,years_to_process
'''data_root="data/"
weather_folder=data_root+"odp/"
gtfs_folder=data_root+"gtfsMavMenetrend/"
gtfs_stops_file="stops.txt"
'''
def get_location_data():
    gtfs_stops=pd.read_csv(gtfs_folder+gtfs_stops_file)
    locs=[]
    passanger_info_folders=[passanger_info_folder+str(y)+"/" for y in years_to_process]
    for l in passanger_info_folders:
        passanger_info_locations_file=l+"t_szolg_helyek.txt"
        places_=pd.read_csv(passanger_info_locations_file,sep=',',encoding='iso-8859-2')
        locs.append(places_)
    places=pd.concat(locs,axis=0)
    for l in locs:
        del l
    #TODO check multiple appearence
    places=places.groupby('TELJES_NEV').agg("last")
    places_with_gtfs=places.merge(gtfs_stops,how='left', left_on='POLGARI_NEV', right_on='stop_name')
    return places_with_gtfs


In [7]:
locations=get_location_data()

In [8]:
locations.head()

,OR_KOD,SZH_KOD,SZH_NEV,POLGARI_NEV,SZH_TIP_KOD,TEHERVONAT,UTASFORGALOM,UZEMVALT,HATARALLOMAS,UZEM_VVALL_VPE_ID,...,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,55,5504135,Aba-Sárkeresztúr,Aba-Sárkeresztúr,1,1.0,1.0,0,0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55,5507229,Abaliget,Abaliget,1,1.0,1.0,0,0,2.0,...,Abaliget,NaN,46.152778,18.073611,NaN,NaN,0.0,NaN,NaN,2.0
2,55,5515610,Abapuszta,Abapuszta,2,0.0,1.0,0,0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55,5513326,Abaújkér,Abaújkér,2,0.0,1.0,0,0,2.0,...,Abaújkér,NaN,48.309444,21.198333,NaN,NaN,0.0,NaN,NaN,2.0
4,55,5513326,Abaújkér,Abaújkér,2,0.0,1.0,0,0,2.0,...,Abaújkér,NaN,48.309444,21.198333,NaN,NaN,0.0,NaN,NaN,2.0


In [9]:
from math import radians, sin, cos, sqrt, atan2

# Function to calculate distance
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers

    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance
'''
# Coordinates of two locations
lat1, lon1 = 52.2296756, 21.0122287  # Warsaw, Poland
lat2, lon2 = 41.8919300, 12.5113300  # Rome, Italy

# Calculate the distance
distance = calculate_distance(lat1, lon1, lat2, lon2)
print(f"The distance is {distance} kilometers.")
'''

'\n# Coordinates of two locations\nlat1, lon1 = 52.2296756, 21.0122287  # Warsaw, Poland\nlat2, lon2 = 41.8919300, 12.5113300  # Rome, Italy\n\n# Calculate the distance\ndistance = calculate_distance(lat1, lon1, lat2, lon2)\nprint(f"The distance is {distance} kilometers.")\n'

In [10]:
weather_meta_file_name=weather_folder+"weather_meta_avg.csv"
met_stat_locations=pd.read_csv(weather_meta_file_name,sep=',',encoding='iso-8859-2')
met_stat_locations

,Loc,Lat,Lon
0,Budapest,47.507300,19.031400
1,Debrecen,47.531438,21.625387
2,Keszthely,46.742617,17.227400
3,Miskolc,48.123882,20.837100
4,Nyiregyhaza,47.969911,21.767344
5,Pecs,46.087188,18.277832
6,Sopron,47.687800,16.603233
7,Szeged,46.251943,20.114371
8,Szombathely,47.249480,16.623060
9,Turkeve,47.105900,20.734733


In [11]:
def get_min_dist_loc(loc_df,lat,lon):
    loc_df['d']=loc_df.apply(lambda x: calculate_distance(lat,lon,x['Lat'],x['Lon']),axis=1)
    return loc_df.loc[loc_df['d'].idxmin()]['Loc']
def get_Locations_coordinates(locations_df,met_locations):
    coord_dict=locations_df[['SZH_KOD','stop_lat','stop_lon']].loc[locations_df['stop_lat'].notna()]
    coord_dict['SZH_KOD']=coord_dict['SZH_KOD'].apply(lambda x: str(x)[2:]).astype(int)
    coord_dict['Legközelebbi állomás']=coord_dict.apply(lambda x: get_min_dist_loc(met_locations,x['stop_lat'],x['stop_lon']),axis=1)
    return coord_dict
coord_dict=get_Locations_coordinates(locations,met_stat_locations)

In [12]:
coord_dict.head()

,SZH_KOD,stop_lat,stop_lon,Legközelebbi állomás
1,7229,46.152778,18.073611,Pecs
3,13326,48.309444,21.198333,Miskolc
4,13326,48.309444,21.198333,Miskolc
5,13300,48.286944,21.181667,Miskolc
6,13318,48.268333,21.179722,Miskolc


In [13]:
'''
event_file_name="vonat_esemenyek_"

events=pd.read_csv(passanger_info_folder+event_file_name+str(2)+'.txt',sep=',',encoding='iso-8859-2')
events.head()
'''

'\nevent_file_name="vonat_esemenyek_"\n\nevents=pd.read_csv(passanger_info_folder+event_file_name+str(2)+\'.txt\',sep=\',\',encoding=\'iso-8859-2\')\nevents.head()\n'

In [14]:
#a=events['IDO'].unique()

In [15]:
#days=pd.to_datetime(events['KOZLEKEDESI_NAP'],format='%Y.%m.%d',errors='coerce')

In [16]:
'''
min_date=days.min()
max_date=days.max()
print(min_date,' - ',max_date)
'''

"\nmin_date=days.min()\nmax_date=days.max()\nprint(min_date,' - ',max_date)\n"

Mit használunk a késésnél előrejelzésnél?
- Szakasz: 
- előző késés
- előző vonat áthaladás
- előző vonat késés 

- szakaszhoz hozzáadjuk pálya állapotot

potenciálisan :
- átlagsebesség a szakaszon



In [17]:
weather_file_name='weather.csv'
import datetime

weather_df=pd.read_csv(weather_folder+weather_file_name,sep=',',encoding='iso-8859-2',parse_dates=['Time'],date_format='%Y-%m-%d')
#weather_df=weather_df.loc[weather_df['Time'].between(min_date,max_date)]


In [18]:
weather_df.head()

,Loc,Time,tx,t,tn,r
0,Budapest,1901-01-01,-0.4,-5.7,-9.2,1.9
1,Budapest,1901-01-02,-6.6,-9.3,-11.3,0.0
2,Budapest,1901-01-03,-6.6,-9.1,-10.8,0.8
3,Budapest,1901-01-04,-9.8,-11.0,-12.4,0.2
4,Budapest,1901-01-05,-9.0,-11.1,-15.5,0.0


In [19]:
collected_trains={}
collected_trains[2]=[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2090, 2103, 2117, 2127, 2171, 2178, 2181, 2188, 2193, 2203, 2204, 2282, 2294, 3210, 3211, 3212, 3213, 3214, 3215, 3216, 3217, 3218, 3219, 3220, 3221, 3222, 3223, 3224, 3226, 3227, 3228, 3229, 3230, 3231, 3232, 3233, 3234, 3235, 3236, 3237, 3238, 3239, 3240, 3241, 3242, 3243, 3244, 3245, 3246, 3247, 3248, 3249, 3250, 3251, 3252, 3253, 3254, 3255, 3256, 3257, 3258, 3259, 3260, 3261, 3262, 3263, 3264, 3265, 3266, 3267, 3268, 3269, 3270, 3272, 3273, 3274, 3275, 3276, 3277, 3278, 3280, 3281, 3282, 3283, 3284, 3285, 3286, 3287]
collected_trains[147]=[17243, 32610, 32612, 32613, 32614, 32615, 32616, 32617, 32619, 32626, 37410, 37412, 37414, 37415, 37416, 37417, 37418, 37419, 37421, 37428, 37431, 37435, 37436, 37438, 37439, 37452, 37454, 37455, 37456, 37457, 37461, 37463, 37465, 37466, 37467]

In [20]:
import pandas as pd
from config import delay_causes_dir
import os

files=os.listdir(delay_causes_dir)
dcr=[]
delay_category_tables=None
def process_delay_causes(train_table,train_event_table,reasoning_table, monitored_trains_array):
    reasons=reasoning_table[["VeID","VkOkSszam","KfkId","SkodId","IgPerc","Indoklas","Vonatszam","VonatDatum"]].copy()
    reasons.rename(columns={'Vonatszam':'OkozoVonat','VonatDatum':'OkozoVonatDatum'},inplace=True)
    train_events=train_event_table[["VeID","VonatID","SzhIndex","SzhKod","EsemID","EsemIdo","KesesPerc"]].copy()
    trains=train_table[['VonatID','vonatszam','vonatdatum']].copy()
    trains.rename(columns={'vonatszam': 'ErintettVonatSzam','vonatdatum':'ErintettVonatDatum'}, inplace=True)
    other_train_caused=reasons[(~reasons["OkozoVonat"].isnull()) & (reasons['KfkId'].isin([6,119]))]
    
    train_caused_reasons_usable=other_train_caused[other_train_caused["OkozoVonat"].isin(monitored_trains_array)]
    full_train_table=train_caused_reasons_usable.merge(train_events,how='left',on='VeID').merge(trains,how='left',on='VonatID')
    full_train_table['OkozoErintettNapKulonbseg']=full_train_table['OkozoVonatDatum']-full_train_table['ErintettVonatDatum']
    train_deps=full_train_table[['OkozoVonat','ErintettVonatSzam','SzhKod','OkozoErintettNapKulonbseg',"KfkId"]].drop_duplicates()
    
    dep_support=full_train_table[['OkozoVonat','ErintettVonatSzam','SzhKod','OkozoErintettNapKulonbseg',"KfkId"]].value_counts()
    
    other_reasons=reasons[~reasons['KfkId'].isin([6,119])]
    train_caused_reasons_unusable=other_train_caused[~other_train_caused["OkozoVonat"].isin(monitored_trains_array)]
    other_usable_reasons=other_reasons[other_reasons["KfkId"].isin([13,110,22,118,188])]
    other_unusable_reasons=other_reasons[~other_reasons["KfkId"].isin([13,110,22,118,188])]
    train_caused_reasons_unusable=train_caused_reasons_unusable.merge(train_events,how='left',on='VeID').merge(trains,how='left',on='VonatID')
    other_unusable_reasons=other_unusable_reasons.merge(train_events,how='left',on='VeID').merge(trains,how='left',on='VonatID')
    to_drop=pd.concat([train_caused_reasons_unusable,other_unusable_reasons],ignore_index=True)
    to_drop=to_drop[['OkozoVonatDatum','ErintettVonatDatum','ErintettVonatSzam']]
    
    return train_deps,dep_support,to_drop
    
#def get_train_deps(train_delays_table):
def get_delay_causes():
    deps={}    
    drops={}
    sups={}
    for f in files:
        if not f.startswith('Vonat'):
            if f.startswith('Torzs'):
                delay_category_tables = pd.read_excel(delay_causes_dir+f,sheet_name=['Esemeny(EsemID)', 'KesesiFokod(KfkID)','KesesiSegedkod(KskID)'])
            print("Ignoring file: {}".format(f))
            continue
        l=f.replace('.xlsx','').split("_")[-1]
        if not l in deps.keys():
            deps[l]=[]
            drops[l]=[]
            sups[l]=[]
        delay_cause_tables = pd.read_excel(delay_causes_dir+f,sheet_name=['Vonat', 'VonatEsemeny','KesesIndoklas'])
        dep_table,sup,to_drop=process_delay_causes(delay_cause_tables['Vonat'],delay_cause_tables['VonatEsemeny'],delay_cause_tables['KesesIndoklas'],collected_trains[int(l)])
        deps[l].append(dep_table)
        drops[l].append(to_drop)
        sups[l].append(sup)
    dependencies={}
    drop_trains={}
    for l in deps.keys():
        dependencies[int(l)]=pd.concat(deps[l],ignore_index=True)
    for l in drops.keys():
        drop_trains[int(l)]=pd.concat(drops[l],ignore_index=True)
    return dependencies,sups,drop_trains

deps,sup,drops=get_delay_causes()

Ignoring file: Torzs.xlsx


C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\821142494.py:18: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  full_train_table['OkozoErintettNapKulonbseg']=full_train_table['OkozoVonatDatum']-full_train_table['ErintettVonatDatum']
C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\821142494.py:18: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  full_train_table['OkozoErintettNapKulonbseg']=full_train_table['OkozoVonatDatum']-full_train_table['ErintettVonatDatum']


Ignoring file: ~$Torzs.xlsx
Ignoring file: ~$Vonat_2022_2.xlsx
Ignoring file: ~$Vonat_2022_2023_147.xlsx


In [21]:
deps[2][deps[2]['OkozoErintettNapKulonbseg']!=pd.Timedelta(0)]

,OkozoVonat,ErintettVonatSzam,SzhKod,OkozoErintettNapKulonbseg,KfkId
227,2010.0,2181,10082,1 days 00:00:00,119
1115,3248.0,3238,10074,1 days 00:00:00,119
1777,2181.0,2188,1453,1 days 00:00:00,119
1859,2082.0,2077,10082,2 days 00:00:00,119
2000,2010.0,2181,10082,1 days 00:00:00,119
2010,2188.0,2181,1453,1 days 00:00:00,119
2143,2074.0,2065,10082,3 days 00:00:00,119
2258,2072.0,2067,10082,1 days 00:00:00,119
2309,2040.0,2019,10082,1 days 00:00:00,119
2496,2010.0,2181,10124,1 days 00:00:00,119


NameError: name 'sup' is not defined

In [4]:
drops[2].shape

NameError: name 'drops' is not defined

In [18]:
deps[2].shape

(3073, 3)

In [19]:
deps[2].head()

,OkozoVonat,ErintettVonatSzam,SzhKod
0,2066.0,2053,10082
1,2066.0,2073,1453
2,3236.0,3233,10124
3,3246.0,3243,10124
4,3246.0,2083,40253


Armstrong's original definition is as follows:

 $$\text{SMAPE} = \frac{1}{n} \sum_{t=1}^n \frac{\left|F_t-A_t\right|}{(A_t+F_t)/2}$$
 miket érdekel az eltérés iránya is  (Teny-Terv)/(Teny+Terv)

In [22]:
from config import event_file_name,passanger_info_folder
import numpy as np


date_format='%Y.%m.%d %H:%M:%S'
drop_cols=['IND_CEL_JELZO','HATARESEM_JELZO','KOFE_IDO','KOFE_MODDAT']+["VSZAM_RENDSZER","EMIG_IDO","EMIG_IDO","EMIG_MODDAT","FOR_IDO","FOR_MODDAT",'ADATJELLEG','LEMONDAS','UTMODDAT',"FOR00_ESEMENY",'VONAL','VONTATO_TIPUS',"UIC","MEGALL_JELZO","UIC","MOZDONYAZONOSITO","EMIG_ESEMENY"]+["ESEMENY_SORSZAM","ALLOMAS_SORSZAM","MEGALLO_SORSZAM"]


# TODO lehet évenként kezelni
# TODO csatlakozás éjjelni vonatnál!! talán nincs ilyen...
def process_events(events,deps,drops):

    events=events.drop(columns=drop_cols)
    events['KOZLEKEDESI_NAP']=pd.to_datetime(events['KOZLEKEDESI_NAP'],format="%Y.%m.%d",errors='coerce')
 
    events['IDO']=pd.to_datetime(events['IDO'],format=date_format,errors='coerce')
    events['TENY_IDO']=pd.to_datetime(events['TENY_IDO'],format=date_format,errors='coerce')
    #events['TENY_IDO'].fillna(events['IDO'],inplace=True)
    events['OSSZ_KESES']=events['TENY_IDO']-events['IDO']

    events['SZH_KOD']=pd.to_numeric(events.apply(lambda x:'%s%s' % (x['ORSZAG_KOD'],x['ALLOMAS']),axis=1))
    
    #esemeny 4 valami olyasmi, hogy megjelnik a vonat/szolgálatba áll
    # 2 indulás
    # 1 érkezés
    # 3 áthaladás
    # 5 kiáll a szolgálatból
    #itt most 4,5 dobható? amikor indulunk, érdekes lehet milyen késéssel tesszük azt. A terv idő itt 0 (4-2 esemény)
    # +vannak olyan állomások, ahol a tervezett tartózkodás 0 
    
    
    events[['ELOZO_OSSZ_KESES','ELOZO_ALLOMAS','ELOZO_ESEMENY_IDO','ELOZO_SZH_KOD']]=events.groupby(['VONATSZAM','KOZLEKEDESI_NAP'])[['OSSZ_KESES','ALLOMAS','IDO','SZH_KOD']].shift()
    events['KESES']=events['OSSZ_KESES']-events['ELOZO_OSSZ_KESES'] 
    events['ELOZO_ESEMENY_IDO'].fillna(events['IDO'],inplace=True)

    events['TERV_IDOTARTAM (m)']= (events['IDO']-events['ELOZO_ESEMENY_IDO']).dt.seconds/60
    events['KESES (m)']=events['KESES']/np.timedelta64(1, 'm')
    events['OSSZ_KESES (m)']=events['OSSZ_KESES']/np.timedelta64(1, 'm')

    events['TENY_IDOTARTAM (m)']=events['TERV_IDOTARTAM (m)']+events['KESES (m)']
    events['RELATIV_KESES']=  (events['TENY_IDOTARTAM (m)']-events['TERV_IDOTARTAM (m)'])/(events['TERV_IDOTARTAM (m)']+events['TENY_IDOTARTAM (m)'])

    events[['ELOZO_SZAKASZ_TERV_IDOTARTAM (m)','ELOZO_SZAKASZ_KESES (m)','ELOZO_SZAKASZ_RELATIV_KESES']]=events.groupby(['VONATSZAM','KOZLEKEDESI_NAP'])[['TERV_IDOTARTAM (m)','KESES (m)','RELATIV_KESES']].shift()

    events['ALLOMAS'] = events['ALLOMAS'].astype(int)
    events['SZH_KOD'] = events['SZH_KOD'].astype(int)

    events['ELOZO_ALLOMAS'].fillna(events['ALLOMAS'],inplace=True)
    #TODO KELL EZ?
    events['ELOZO_SZH_KOD'].fillna(events['SZH_KOD'],inplace=True)

    events['ID']=  events['ELOZO_ALLOMAS'].astype(int).astype(str)+"-"+events['ALLOMAS'].astype(str)

    
    deps['ID']=deps['SzhKod'].astype(str)+'-'+deps['SzhKod'].astype(str)
    deps.rename(columns={'ErintettVonatSzam':"VONATSZAM","ErintettVonatDatum":"KOZLEKEDESI_NAP"},inplace=True)
    deps['OkozoVonat']=deps['OkozoVonat'].astype(int)
    events=events.merge(deps,on=['ID','VONATSZAM'],how='left')
    # beérés késése
    events_deps=events[["VONATSZAM","TENY_IDO","KOZLEKEDESI_NAP","ALLOMAS","ELOZO_ALLOMAS"]][(events['ALLOMAS']!=events['ELOZO_ALLOMAS'])]        
    events=events.merge(events_deps,left_on=['SzhKod','OkozoVonat','KOZLEKEDESI_NAP'],right_on=['ALLOMAS','VONATSZAM','KOZLEKEDESI_NAP'],how='left')
    #hogyan viszonyul a késés tenyidő a TERVEZETT induláshoz (IDO)?
    events['CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST']=events['TENY_IDO_y']-events['IDO']
    #negativ= nem kesett
    #nagyon negativ egy nappal elobbi TENY_IDO_y - adjunk hozza napokat amig nem nagyon negativa
    #pozitiv = kesett
    #nagyon pozitiv egy nappal keseobbi TENY_IDO_y -vonajunk le
    def fix_delay(difference:pd.Timedelta):
        while(difference<-pd.Timedelta(720,'m')):
            difference+=pd.Timedelta(1,'d')
        while(difference>pd.Timedelta(720,'m')):
            difference-=pd.Timedelta(1,'d')
        return difference
    #events['KORRIGÁLT']=events['CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST'].map(fix_delay)
    events['CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST']=events['CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST'].map(fix_delay)
    events['CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST'].fillna(pd.Timedelta(0),inplace=True)
    #events['CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST']=events['CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST'] if events['CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST']>pd.Timedelta(0) else pd.Timedelta(0)
    events['CSATL_MAX_KESES (m)']=events.groupby(['KOZLEKEDESI_NAP','VONATSZAM_x','ID'])['CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST'].transform(max)
    events['CSATL_MAX_KESES (m)']=events['CSATL_MAX_KESES (m)'].dt.total_seconds()/60

    
    #itt van egy kis probléma.. lehet hogy a csatlakozó vonat közlekedési napja más...
    #˘1440 perc körüli késés...
    # HACK: mondjuk 1000 felett akkor -1440? 
    #tehát más közlekedési napú vonatra várunk - közl nap + vonatid a kér vonatdatum nem fog egyezni a keses ok táblákban
    
    drops.rename(columns={'ErintettVonatDatum':'KOZLEKEDESI_NAP','ErintettVonatSzam':'VONATSZAM'},inplace=True)
    #TODO miért kell most?
    drops['KOZLEKEDESI_NAP']=pd.to_datetime(drops['KOZLEKEDESI_NAP'])
    drops.rename(columns={"VONATSZAM":"VONATSZAM_x"},inplace=True)
    events=events.merge(drops.drop_duplicates(), on=['KOZLEKEDESI_NAP','VONATSZAM_x'], 
                   how='left', indicator=True)
    print("Before drop: ",events.shape)
    events=events[events['_merge'] == 'left_only']
    events.drop(columns=['_merge'],inplace=True)
    
    print("After drop: ",events.shape)
    
    rename_after_merges={"VONATSZAM_x":"VONATSZAM","TENY_IDO_x":"TENY_IDO","ALLOMAS_x":"ALLOMAS"}

    delete_after_merges=["ALLOMAS_y","TENY_IDO_y","OkozoVonat",'ALLOMAS_y','TENY_IDO_y','ELOZO_ALLOMAS_y','CSATLAKOZAS_KESESE_INDULASHOZ_KEPEST','KfkId','SzhKod','VONATSZAM_y',"ORSZAG_KOD","ELOZO_ALLOMAS_x","ELOZO_SZH_KOD","OkozoErintettNapKulonbseg","OkozoVonatDatum","OkozoErintettNapKulonbseg"] 
    events.rename(columns=rename_after_merges,inplace=True)
    events.drop(columns=delete_after_merges,inplace=True)
    #events.drop_duplicates(inplace=True)
    

    #itt fontos lehet a dátum is, ezért érdemes az időt is megtartani
    # viszont az idő inkább a napszak miatt érdekes
    events['MENETREND_IDO (m)']=events['IDO'].dt.hour*60+events['IDO'].dt.minute
    #events['MENETREND_IDO (m)']=events['MENETREND_IDO (m)'].dt.total_seconds()/60
    events=events[events['ESEMENY']!=4]
    events.drop(columns=['ESEMENY','ELOZO_OSSZ_KESES','KESES','ELOZO_OSSZ_KESES','OSSZ_KESES'],inplace=True)
    events=events.fillna(0.0)
    

    min_date=events['KOZLEKEDESI_NAP'].min()
    max_date=events['KOZLEKEDESI_NAP'].max()
    print(min_date,' - ',max_date)
    return events,min_date,max_date

def add_weather_data(coord_dict,weather_df,events):
    #events = e.copy()
    events['IndulóÁllomás']=events.groupby(['VONATSZAM','KOZLEKEDESI_NAP']).first().reset_index()['ALLOMAS'].astype(int)
    events=events.merge(coord_dict, how='left', left_on='IndulóÁllomás',right_on='SZH_KOD')
    events = events.merge(weather_df, how='left', left_on=['Legközelebbi állomás', 'KOZLEKEDESI_NAP'],
                              right_on=['Loc', 'Time'])
    return events

def get_events_data(weather_df,coord_dict,deps,drops,year=2023,line=147 ):
    l= passanger_info_folder + str(year) + "/"
    e=pd.read_csv(l+event_file_name+str(line)+'.txt',sep=',',encoding='iso-8859-2')
    
    e,min_date,max_date=process_events(e,deps[line],drops[line])
    weather_df=weather_df.loc[weather_df['Time'].between(min_date,max_date)]

    e=add_weather_data(coord_dict,weather_df,e)
    junk_from_joins=['Loc','SZH_KOD_x','SZH_KOD_y','stop_lat','stop_lon','Time','Legközelebbi állomás','IndulóÁllomás','ALLOMAS']
    
    
    e.drop(columns=junk_from_joins,inplace=True)
    e['LINE']=line
    return e

l= passanger_info_folder + str(2022) + "/"
e=pd.read_csv(l+event_file_name+str(2)+'.txt',sep=',',encoding='iso-8859-2')
res=process_events(e,deps[2],drops[2])



#events_2=get_events_data(weather_df,coord_dict,deps, drops)  



C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\942977587.py:145: DtypeWarning: Columns (16,17,28) have mixed types. Specify dtype option on import or set low_memory=False.
  e=pd.read_csv(l+event_file_name+str(2)+'.txt',sep=',',encoding='iso-8859-2')
C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\942977587.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  events['ELOZO_ESEMENY_IDO'].fillna(events['IDO'],inplace=True)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\942977587.py:48: FutureWarning: A value is trying to be

Before drop:  (3855893, 35)
After drop:  (2789964, 34)


C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\942977587.py:113: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  events=events.fillna(0.0)


2022-01-01 00:00:00  -  2023-12-31 00:00:00


In [33]:
res[0][res[0]['CSATL_MAX_KESES (m)']>0.0].head(10)

,VONATSZAM,KOZLEKEDESI_NAP,ALLOMAS,IDO,TENY_IDO,SZH_KOD,ELOZO_ESEMENY_IDO,TERV_IDOTARTAM (m),KESES (m),OSSZ_KESES (m),TENY_IDOTARTAM (m),RELATIV_KESES,ELOZO_SZAKASZ_TERV_IDOTARTAM (m),ELOZO_SZAKASZ_KESES (m),ELOZO_SZAKASZ_RELATIV_KESES,ID,CSATL_MAX_KESES (m),MENETREND_IDO (m)
4,2010,2022-01-01,10124,2022-01-01 00:36:00,2022-01-01 00:36:09,5510124,2022-01-01 00:35:00,1.0,0.233333,0.150000,1.233333,0.104478,5.0,-0.750000,-0.081081,10124-10124,1.750000,36.0
145,2029,2022-01-01,1487,2022-01-01 04:36:00,2022-01-01 04:36:45,551487,2022-01-01 04:35:00,1.0,-0.383333,0.750000,0.616667,-0.237113,4.0,0.033333,0.004149,1487-1487,23.800000,276.0
153,2029,2022-01-01,1453,2022-01-01 04:47:00,2022-01-01 04:48:31,551453,2022-01-01 04:46:00,1.0,-0.333333,1.516667,0.666667,-0.200000,2.0,-0.316667,-0.085973,1453-1453,0.833333,287.0
154,2029,2022-01-01,1453,2022-01-01 04:47:00,2022-01-01 04:48:31,551453,2022-01-01 04:46:00,1.0,-0.333333,1.516667,0.666667,-0.200000,2.0,-0.316667,-0.085973,1453-1453,0.833333,287.0
209,2030,2022-01-01,1438,2022-01-01 04:34:00,2022-01-01 04:35:46,551438,2022-01-01 04:33:00,1.0,-0.416667,1.766667,0.583333,-0.263158,2.0,-0.450000,-0.126761,1438-1438,0.700000,274.0
210,2030,2022-01-01,1438,2022-01-01 04:34:00,2022-01-01 04:35:46,551438,2022-01-01 04:33:00,1.0,-0.416667,1.766667,0.583333,-0.263158,2.0,-0.450000,-0.126761,1438-1438,0.700000,274.0
218,2030,2022-01-01,1453,2022-01-01 04:47:00,2022-01-01 04:48:23,551453,2022-01-01 04:46:00,1.0,-0.450000,1.383333,0.550000,-0.290323,3.0,-0.633333,-0.118012,1453-1453,0.850000,287.0
219,2030,2022-01-01,1453,2022-01-01 04:47:00,2022-01-01 04:48:23,551453,2022-01-01 04:46:00,1.0,-0.450000,1.383333,0.550000,-0.290323,3.0,-0.633333,-0.118012,1453-1453,0.850000,287.0
220,2030,2022-01-01,1453,2022-01-01 04:47:00,2022-01-01 04:48:23,551453,2022-01-01 04:46:00,1.0,-0.450000,1.383333,0.550000,-0.290323,3.0,-0.633333,-0.118012,1453-1453,0.850000,287.0
239,2050,2022-01-01,10124,2022-01-01 05:21:00,2022-01-01 05:23:49,5510124,2022-01-01 05:20:00,1.0,-0.566667,2.816667,0.433333,-0.395349,5.0,-1.016667,-0.113173,10124-10124,1.966667,321.0


In [35]:
#events_2.head(50)


,VONATSZAM,KOZLEKEDESI_NAP,ALLOMAS,IDO,TENY_IDO,TERV_IDOTARTAM (m),KESES (m),OSSZ_KESES (m),TENY_IDOTARTAM (m),RELATIV_KESES,ELOZO_SZAKASZ_TERV_IDOTARTAM (m),ELOZO_SZAKASZ_KESES (m),ELOZO_SZAKASZ_RELATIV_KESES,ID,CSATL_MAX_KESES (m),MENETREND_IDO (m),tx,t,tn,r
0,32613,2023-01-01,18572,2023-01-01 15:46:00,2023-01-01 15:46:17,0.0,0.283333,0.283333,0.283333,1.000000,0.0,0.000000,0.000000,18572-18572,0.000000,946,15.1,8.2,2.9,0.0
1,32613,2023-01-01,18564,2023-01-01 15:50:00,2023-01-01 15:50:26,4.0,0.150000,0.433333,4.150000,0.018405,0.0,0.283333,1.000000,18572-18564,0.000000,950,15.1,8.2,2.9,0.0
2,32613,2023-01-01,18564,2023-01-01 15:55:00,2023-01-01 15:56:43,5.0,1.283333,1.716667,6.283333,0.113737,4.0,0.150000,0.018405,18564-18564,0.000000,955,15.1,8.2,2.9,0.0
3,32613,2023-01-01,18226,2023-01-01 16:04:00,2023-01-01 16:05:50,9.0,0.116667,1.833333,9.116667,0.006440,5.0,1.283333,0.113737,18564-18226,0.000000,964,15.1,8.2,2.9,0.0
4,32613,2023-01-01,18226,2023-01-01 16:05:00,2023-01-01 16:06:16,1.0,-0.566667,1.266667,0.433333,-0.395349,9.0,0.116667,0.006440,18226-18226,0.000000,965,15.1,8.2,2.9,0.0
5,32613,2023-01-01,18218,2023-01-01 16:11:00,2023-01-01 16:12:18,6.0,0.033333,1.300000,6.033333,0.002770,1.0,-0.566667,-0.395349,18226-18218,0.000000,971,15.1,8.2,2.9,0.0
6,32613,2023-01-01,18218,2023-01-01 16:12:00,2023-01-01 16:12:51,1.0,-0.450000,0.850000,0.550000,-0.290323,6.0,0.033333,0.002770,18218-18218,0.000000,972,15.1,8.2,2.9,0.0
7,32613,2023-01-01,44503,2023-01-01 16:17:00,2023-01-01 16:17:35,5.0,-0.266667,0.583333,4.733333,-0.027397,1.0,-0.450000,-0.290323,18218-44503,0.000000,977,15.1,8.2,2.9,0.0
8,32613,2023-01-01,44503,2023-01-01 16:17:00,2023-01-01 16:17:55,0.0,0.333333,0.916667,0.333333,1.000000,5.0,-0.266667,-0.027397,44503-44503,0.000000,977,15.1,8.2,2.9,0.0
9,32613,2023-01-01,18200,2023-01-01 16:20:00,2023-01-01 16:21:01,3.0,0.100000,1.016667,3.100000,0.016393,0.0,0.333333,1.000000,44503-18200,0.000000,980,15.1,8.2,2.9,0.0


In [36]:
deps[147].head()

,OkozoVonat,VONATSZAM,SzhKod,ID
0,32619,32610,18184,18184-18184
1,32613,32616,18184,18184-18184
2,37467,32617,18184,18184-18184
3,32615,32614,18184,18184-18184
4,37431,37428,17137,17137-17137


In [39]:
import numpy as np
#elegánsabb lenne VONATSZAM és VONAL nélkül, de ahhot több adat kell majd..
# ELOZO_ESEMENY_IDO a line status joinhoz -terv
#TODO  LINE egy közelítés az állapot leírására, ezen majd lehet dolgozni azért...
#TODO ELOZO_KESES elkodolhat maradek információkat ezen is lehet majd dolgozni, bár rendenkezésre állhat
#TODO VONATSZAM szintén menetrendi, prioritás információk, de túl sok lehetőség,és ellentmond a logikánknak inkább érdemes dobni
#érdekes lehet : MOZDONYAZONOSITO, UIC, VONTATO_TIPUS
#to_keep=['VONAL','VONATSZAM','KOZLEKEDESI_NAP','VONTATO_TIPUS','IDO','ELOZO_ESEMENY_IDO','MENETREND_IDO','OSSZ_KESES (m)','ELOZO_SZAKASZ_KESES (m)','KESES (m)','RELATIV_KESES','TERV_IDOTARTAM (m)','CSATL_MAX_KESES','tx','t','tn','r']
#def get_events_pred_data(processed, line=2):
    #ezek azok amik kellenek most, és később is
    #processed=processed[to_keep]
    #processed['LINE']=line
    #return processed
#d2=get_events_pred_data(events_2,147)
#d2.to_pickle(data_root+'test_events_data.pkl')


# Vonatfej - TODO
nem tudom mire lesz még jó..

In [ ]:
#TODO 
def get_train_head(line=2):
    heads=[]
    for l in  [passanger_info_folder+str(y)+"/" for y in years_to_process]:
        fn = l + f'vonat_fej_{str(line)}.txt'
        h = pd.read_csv(fn,sep=',',encoding='iso-8859-2')
        heads.append(h)
    train_head=pd.concat(heads,axis=0)    
    train_head['ERK_IDO_MN']=pd.to_datetime(train_head['ERK_IDO'],format='%Y.%m.%d %H:%M:%S').dt.time
    train_head['IND_IDO_MN']=pd.to_datetime(train_head['IND_IDO'],format='%Y.%m.%d %H:%M:%S').dt.time
    return train_head

head_2=get_train_head()

In [ ]:
head_2.head()

In [ ]:
def get_different_heads(train_head,train_id):
    if train_id:
        train_head=train_head.loc[train_head['VONATSZAM']==train_id]
    train_head['KOZLEKEDESI_NAP_DT']=pd.to_datetime(train_head['IND_IDO'],format='%Y.%m.%d %H:%M:%S').dt.date
    separate_trains_in_schedule=train_head[['VONATSZAM','VNEM_KOD','IND_ALL_KOD','CEL_ALL_KOD','CEL_ESEMENY_SORSZAM','KOZLEKEDESI_NAP_DT']].drop_duplicates()
    route_start_ends= separate_trains_in_schedule.groupby(['VONATSZAM','IND_ALL_KOD','CEL_ALL_KOD','CEL_ESEMENY_SORSZAM'])['KOZLEKEDESI_NAP_DT'].agg(first="min",last="max",count="count")
    return route_start_ends
def get_different_schedules(head,events,train_id=None):
    train_event_data=events
    if train_id:
        train_event_data=train_event_data.loc[train_event_data['VONATSZAM']==train_id]
    route_start_ends = get_different_heads(head,train_id)
    schedule_variants=train_event_data.loc[train_event_data['KOZLEKEDESI_NAP_DT'].isin(route_start_ends['first'])]
    return schedule_variants

    
    
    

variants = get_different_schedules(head_2,events_2,train_id=2010)    

# Menetrend variációk Vizualizációja

In [ ]:
import datetime

variants = variants[['KOZLEKEDESI_NAP_DT', 'ELOZO_ALLOMAS', 'ELOZO_ESEMENY_IDO', 'ALLOMAS', 'MENETREND_IDO']].dropna()
variants['ELOZO_ESEMENY_IDO']=variants['ELOZO_ESEMENY_IDO'].dt.time
# todo astype to creation

variants.head()

In [ ]:

today_ = datetime.date.today()
variants['Start'] = variants.apply(
    lambda x: datetime.datetime.combine(today_, x['ELOZO_ESEMENY_IDO']), axis=1)
variants['Finish'] = variants.apply(
    lambda x: datetime.datetime.combine(today_, x['MENETREND_IDO']), axis=1)
variants['Task'] = variants['ELOZO_ALLOMAS'].astype(str) + '-' + \
                                     variants['ALLOMAS'].astype(str)



In [ ]:
import plotly.express as px

fig = px.timeline(variants, x_start="Start", x_end="Finish", y="KOZLEKEDESI_NAP_DT", color="Task")
fig.update_yaxes(autorange="reversed", dtick=1)

fig.show()

### Data Preprocessing 

# Lassújel és vágányzár

In [ ]:
import pandas as pd

slow_signal_tables = pd.read_excel('data/obstacles/lassújel_2_147_vonal_20220101-20231231_AI-hoz.xlsx',
                                   sheet_name=['ZVZ_KEIHEAD', 'ZVZ_KIZARASOK', 'ZVZ_VZTHEAD'])
closure_tables = pd.read_excel('data/obstacles/vágányzár_2_147_vonal_20220101-20231231_AI-hoz.xlsx    ',
                               sheet_name=['ZVZ_KIZARASOK', 'ZVZ_VZTHEAD'])


In [ ]:
ss_parts=slow_signal_tables['ZVZ_KIZARASOK'].copy()
ss_parts.drop(axis=0,index=0,inplace=True)


ss_parts['Kizárásszám']=ss_parts['Kizárásszám'].astype(str)


to_drop=['Kizárás tétel','Kizárás típusa','Kezdő SZH statisztikai szám','Záró SZH statisztikai szám','Lassúmenet kezdő szelvény','Lassúmenet záró szelvény','Változó munkahelyen kitűzött','Lassúmenet mértéke tehervonatra (km/h)','Kis teljesítményű dízel kitoló','Dízel kitoló szükséges','Nagy teljesítményű dízel kitoló', 'Állomási bizt.ber. kikapcsolás szükséges','Vonali bizt. ber. kikapcsolás szükséges',"INF vonal","INF vonal.1","HÜSZ vonal","HÜSZ vonal.1"]
#todo később jól jöhetnek: 
# Lassúmenet kezdő szelvény, Lassúmenet záró szelvény,Lassúmenet mértéke tehervonatra (km/h),Kis teljesítményű dízel kitoló (csupa 0),Dízel kitoló szükséges (nincs ilyen),Nagy teljesítményű dízel kitoló (nincs ilyen),Állomási bizt.ber. kikapcsolás szükséges(nincs ilyen)
# kérdéses: Változó munkahelyen kitűzött, Kizárás típusa (redundáns?)
#todo kezdő, záró szolgálati hely mindig egy minimális szakasz?
cols= ss_parts.columns
for d in to_drop:
    if d not in cols:
        print('Missing:',d)
    
ss_parts.drop(columns=to_drop,inplace=True)

#lassúmenet iránya egy adott sínpárra/vágányra vonatkozik, de sínpár is lehet
# Vágányzár VZ írja le melyik vágányról beszélünk.

# ha > akkor a jobb oldali lehet érdekesebb 
#ss_parts['S_ID']=ss_parts.apply(lambda x: transform_SZH_KOD(x['Kezdő szolgálati hely'],x['Záró szolgálati hely']) if ">" in x["Lassúmenet iránya"] else -1   ,axis=1)

# ha  < akkor a bal 
#ss_parts['S_ID_R']=ss_parts.apply(lambda x: transform_SZH_KOD(x['Záró szolgálati hely'],x['Kezdő szolgálati hely']) if "<" in x["Lassúmenet iránya"] else -1 ,axis=1)

# kellene olyasmi hogy Irány szerinti oldal, és ellentétes oldal lezárva ehhez érdemes lehet kettészedni az irányokat..
# todo: VZ van jobb/bal oldali közlekedés? most abból indulunk ki, hogy van valmilyen szinten azaz szakszmenti és ellentétes - J: I(rány) a nem fordítottnál, E(llentétes) a fordítottnál, B  E(llenttes) a nem fordítottnál és I a fordítottnál

#todo VZ elméletben lehet A is, gyakorlatban csak 1 és 4 van
ss_parts.loc[ss_parts['Záró szolgálati hely'].isna(),'Záró szolgálati hely']=ss_parts['Kezdő szolgálati hely']

ss_parts_fw = ss_parts[ss_parts['Lassúmenet iránya'].str.contains(">")]
ss_parts_bw = ss_parts[ss_parts['Lassúmenet iránya'].str.contains("<")]

def set_section_id(df):
    df['Kezdő szolgálati hely']=df['Kezdő szolgálati hely'].str.replace('M-55','')
    df['Záró szolgálati hely']=df['Záró szolgálati hely'].str.replace('M-55','')
    
    df['ID']=df['Kezdő szolgálati hely'].astype(str)+"-"+df['Záró szolgálati hely'].astype(str)

def encode_directions(dff,dfb):
    dff["I"]=dff.apply(lambda x: 1 if x['Vágány (VZ)'] in [1,3,4]  else 0,axis=1)
    dff["E"]=dff.apply(lambda x: 1 if x['Vágány (VZ)'] in [2,3,4]  else 0,axis=1)
    dff["ÁI"]=dff.apply(lambda x: 1 if x['Átmenő vágány típusa'] in ['A','Á','J']  else 0,axis=1)
    dff["ÁE"]=dff.apply(lambda x: 1 if x['Átmenő vágány típusa'] in ['A','Á','B']  else 0,axis=1)
    
    dfb["I"]=dfb.apply(lambda x: 1 if x['Vágány (VZ)'] in [2,3,4]  else 0,axis=1)
    dfb["E"]=dfb.apply(lambda x: 1 if x['Vágány (VZ)'] in [1,3,4]  else 0,axis=1)
    dfb["ÁI"]=dfb.apply(lambda x: 1 if x['Átmenő vágány típusa'] in ['A','Á','B']  else 0,axis=1)
    dfb["ÁE"]=dfb.apply(lambda x: 1 if x['Átmenő vágány típusa'] in ['A','Á','J']  else 0,axis=1)
    dfb[['Kezdő szolgálati hely','Záró szolgálati hely']]=dfb[['Záró szolgálati hely','Kezdő szolgálati hely']]
    return pd.concat([dff,dfb],axis=0)


ss_parts=encode_directions(ss_parts_fw,ss_parts_bw)
set_section_id(ss_parts)

ss_parts = ss_parts.drop(columns=['Kezdő szolgálati hely','Záró szolgálati hely','Lassúmenet iránya','Vágány (VZ)','Átmenő vágány típusa']) 

ss_parts.head()

In [ ]:
# csak lassújelnél
# helyettesítsük 0-val VZ-nál?
ss_parts['Lassúmenet mértéke motorvonatra (km/h)']=ss_parts['Lassúmenet mértéke motorvonatra (km/h)'].fillna(ss_parts['Lassúmenet mértéke (km/h)'])
#todo pontosítás lassúmenet mekkora része az adott szakasznak? - A tervidő segíthet
#todo milyen vonat?? motor? 

In [ ]:
import numpy as np
# igen/nem
ss_parts['Fels.vez. kikapcsolása']=ss_parts['Fels.vez. kikapcsolása'].map({np.nan:0,'X':1})
# ideiglenes az, ami hatással lehet a késésre, elvileg az állandónak már benne kellene lenni a menetrendben 
ss_parts['Lassúmenet típusa']=ss_parts['Lassúmenet típusa'].map({'Á':0,'I':1})
#ss_parts=pd.get_dummies(ss_parts, columns=['Érintett objektum típusa'],dtype=int)


In [ ]:
ss_status = slow_signal_tables['ZVZ_KEIHEAD'].copy()

ss_status.drop(axis=0,index=0,inplace=True)
ss_status['Vágányzár KEI ID']=ss_status['Vágányzár KEI ID'].astype(str).str.replace('.0','')
#ss_status['Utolsó jóváhagyott kezdés dátuma']=pd.to_datetime( ss_status['Utolsó jóváhagyott kezdés dátuma'],format='%Y-%m-%d').dt.date    
#ss_status['Utolsó jóváhagyott kezdés időpontja']=pd.to_datetime( ss_status['Utolsó jóváhagyott kezdés időpontja']).dt.time

#excel típus információ lehet, alapból dátum datetime64[ns] lesz, így visszakovert után formátum változik
ss_status['Kezdés']= pd.to_datetime(ss_status['Utolsó jóváhagyott kezdés dátuma'].astype(str)+' '+ss_status['Utolsó jóváhagyott kezdés időpontja'].astype(str),format='%Y-%m-%d %H:%M:%S')
#ss_status['Utolsó jóváhagyott befejezés dátuma']=pd.to_datetime( ss_status['Utolsó jóváhagyott befejezés dátuma'],format='%Y-%m-%d').dt.date    
#ss_status['Utolsó jóváhagyott befejezés időpontja']=pd.to_datetime( ss_status['Utolsó befejezés kezdés időpontja']).dt.time

#
ss_status['Befejezés']= pd.to_datetime(ss_status['Utolsó jóváhagyott befejezés dátuma'].astype(str)+' '+ss_status['Utolsó jóváhagyott befejezés időpontja'].astype(str),format='%Y-%m-%d %H:%M:%S',errors='coerce')


'''
ss_status['Tény-indítás (dátum)']=pd.to_datetime( ss_status['Tény-indítás (dátum)'],format='%Y-%m-%d').dt.date   
ss_status['TényIndít.időpontja']=pd.to_datetime( ss_status['TényIndít.időpontja']).dt.time
ss_status['Tény-befejez.(dátum)']=pd.to_datetime( ss_status['Tény-befejez.(dátum)'],format='%Y-%m-%d').dt.date   
ss_status['Tény-befej.ideje']=pd.to_datetime( ss_status['Tény-befej.ideje']).dt.time
'''

to_drop_head=['Státusséma','Utolsó jóváhagyott kezdés dátuma','Utolsó jóváhagyott kezdés időpontja','Utolsó jóváhagyott befejezés dátuma','Utolsó jóváhagyott befejezés időpontja','Státusséma','Vágányzár Státus','Korlátozás típus']
ss_status=ss_status.drop(columns=to_drop_head)


In [ ]:
merged=ss_parts.merge(ss_status,how='left',left_on='Kizárásszám',right_on='Vágányzár KEI ID')


In [ ]:
c_head=closure_tables['ZVZ_VZTHEAD'].copy()
# "lemondott" -  akkor ez nem történt meg?
canceled=['KNJM','LEMD','LEMM','LEMO','KELU']
# "törölt" ez mit jelent ? szintén nem történt meg?   
deleted=['ELUT','VPEE','VTÖR','KELU']
not_valid=canceled+deleted
c_head['Státus'] = 1
c_head.loc[c_head['Vágányzár Státus'].isin(not_valid), "Státus"] = 0

c_head['Kiutalt menetvonalak zavartatása nélkül'] = c_head['Kiutalt menetvonalak zavartatása nélkül'].map(
    {np.nan: 0, 'X': 1})
c_head.drop(axis=0,index=0,inplace=True)
c_head['Vágányzár VZT ID']=c_head['Vágányzár VZT ID'].astype(str).str.replace('.0','')
#ss_status['Utolsó jóváhagyott kezdés dátuma']=pd.to_datetime( ss_status['Utolsó jóváhagyott kezdés dátuma'],format='%Y-%m-%d').dt.date    
#ss_status['Utolsó jóváhagyott kezdés időpontja']=pd.to_datetime( ss_status['Utolsó jóváhagyott kezdés időpontja']).dt.time

#excel típus információ lehet, alapból dátum datetime64[ns] lesz, így visszakovert után formátum változik
c_head['Kezdés']= pd.to_datetime(c_head['Tény-indítás (dátum)'].astype(str)+' '+c_head['TényIndít.időpontja'].astype(str),format='%Y-%m-%d %H:%M:%S',errors='coerce')
#ss_status['Utolsó jóváhagyott befejezés dátuma']=pd.to_datetime( ss_status['Utolsó jóváhagyott befejezés dátuma'],format='%Y-%m-%d').dt.date    
#ss_status['Utolsó jóváhagyott befejezés időpontja']=pd.to_datetime( ss_status['Utolsó befejezés kezdés időpontja']).dt.time

#
c_head['Befejezés']= pd.to_datetime(c_head['Tény-befejez.(dátum)'].astype(str)+' '+c_head['Tény-befej.ideje'].astype(str),format='%Y-%m-%d %H:%M:%S',errors='coerce')
#ementrend szempontjából nem érdekes
to_drop_c_head=['Korlátozás típus','Korlátozottan.rend.álló kap.','Státusséma','Vágányzár Státus','Tény-indítás (dátum)','TényIndít.időpontja','Tény-befejez.(dátum)','Tény-befej.ideje','Utolsó jóváhagyott befejezés dátuma','Utolsó jóváhagyott befejezés időpontja','Utolsó jóváhagyott kezdés dátuma','Utolsó jóváhagyott kezdés időpontja','Vágányzár túllépve','Van vonatkésés','Pótolni kell']

c_head.drop(columns=to_drop_c_head,inplace=True)

In [ ]:
c_parts = closure_tables['ZVZ_KIZARASOK'].copy()

c_parts.loc[c_parts['Záró szolgálati hely'].isna(), 'Záró szolgálati hely'] = c_parts['Kezdő szolgálati hely']
c_parts_bw=c_parts.copy()



#TODO  Vágány (VZ) és 'Átmenő vágány típusa' ugyanannak néz ki mint lassújelnél, ugyanazt a logikát csináljuk itt is 
#itt vannak 1,2,3,4 értékek VZ-nél, elvileg lehet 'A' itt is 

c_parts=encode_directions(c_parts,c_parts_bw)
set_section_id(c_parts)

c_parts = c_parts.drop(columns=['Kezdő szolgálati hely','Záró szolgálati hely','Vágány (VZ)','Átmenő vágány típusa']) 
to_drop_gen=['Kizárás tétel','Kizárás típusa','Kezdő SZH statisztikai szám','Záró SZH statisztikai szám','Kis teljesítményű dízel kitoló','Dízel kitoló szükséges','Nagy teljesítményű dízel kitoló', 'Állomási bizt.ber. kikapcsolás szükséges','Vonali bizt. ber. kikapcsolás szükséges',"INF vonal","INF vonal.1","HÜSZ vonal","HÜSZ vonal.1"]

to_drop_ss=['Lassúmenet kezdő szelvény','Lassúmenet záró szelvény','Változó munkahelyen kitűzött','Lassúmenet mértéke tehervonatra (km/h)']
c_parts['Fels.vez. kikapcsolása']=c_parts['Fels.vez. kikapcsolása'].map({np.nan:0,'X':1})

c_parts.drop(columns=to_drop_gen,inplace=True)

#TODO ezen azért lehet majd még gondolkodni...
#c_parts['Lassúmenet mértéke (km/h)']=0.0
#c_parts['Lassúmenet mértéke motorvonatra (km/h)']=0.0
#TODO ez lehet gáz
#c_parts['Lassúmenet hossza']=0
#c_parts['Lassúmenet típusa']=1

c_parts['Kizárásszám']=c_parts['Kizárásszám'].astype(str).str.replace('.0','')

c_merged=c_parts.merge(c_head,how='left',left_on='Kizárásszám',right_on='Vágányzár VZT ID')



In [ ]:
merged['Kizárás (1=VZ,2=LJ)']=2
c_merged['Kizárás (1=VZ,2=LJ)']=1

In [ ]:
line_status=pd.concat([merged,c_merged])
del merged,c_merged
line_status.head()

In [ ]:
final_drop=['Vágányzár KEI ID','Vágányzár VZT ID']
line_status.drop(columns=final_drop,inplace=True)

In [ ]:
line_status.fillna(0.0,inplace=True)


In [ ]:
line_status.head()

In [40]:
fn='data/line_status.csv'
line_status.to_pickle(fn)
del line_status

# load
df = pd.read_pickle(fn)

NameError: name 'line_status' is not defined

In [ ]:
df.head()

# Végső összerakás

In [23]:
def add_line_info(e, line_info):
    #e=events.copy()
    #d2.reset_index(inplace=True)
    joined = e.merge(line_info,on='ID',how='inner')
    # todo miért kell???
    joined['Kezdés']=pd.to_datetime(joined['Kezdés'])
    joined['Befejezés']=pd.to_datetime(joined['Befejezés'])
    #ezek azok az események, ahol volt valami korlátozás
    joined = joined[joined['ELOZO_ESEMENY_IDO'].between(joined['Kezdés'],joined['Befejezés'])]
    e=e.reindex(columns=joined.columns)
    e.update(joined)
    e.fillna(0.0,inplace=True)
    e.drop(columns='ELOZO_ESEMENY_IDO',inplace=True)
    return e



fn='data/line_status.csv'
line_status = pd.read_pickle(fn)
#events_with_line_info=add_line_info(d2, line_status)
#events_with_line_info.head()
    

In [45]:
# https://stackoverflow.com/questions/50422404/save-and-export-dtypes-information-of-a-python-pandas-dataframe
def save_data_with_types(df,fn):
    with open(fn, 'wt') as f:
        f.write(',' + ','.join(map(str, df.dtypes)) + '\n')
        df.to_csv(f, lineterminator='\n')

def load_data(fn): 
    with open(fn, 'rt') as f:
        types = next(f).rstrip().split(',')[1:]
        columns = next(f).rstrip().split(',')[1:]
        df = pd.read_csv(f, dtype=dict(zip(columns, types)), index_col=0, names=columns)
        return df

In [26]:
def add_previous_train_delay(e):
    #e=events.copy()
    #TODO érdemes lehet mozdonyid-ra is csoportosítani
    e[['ELOZO_VONAT_KESES (m)','ELOZO_STATUS','ELOZO_VONAT_RELATIV_KESES']]=e.groupby('ID')[['KESES (m)','Kizárásszám','RELATIV_KESES']].shift()
    e[['ELOZO_VONAT_KESES (m)','ELOZO_STATUS','ELOZO_VONAT_RELATIV_KESES']].fillna(0.0,inplace=True)
    e['VONAL_STATUSZ_VALTOZOTT']=e['ELOZO_STATUS']!=e['Kizárásszám']
    e.drop(columns=['ELOZO_STATUS','Kizárásszám'],inplace=True)
    return e

#events_with_line_info=add_previous_train_delay(events_with_line_info)

In [15]:
#events_with_line_info.loc[events_with_line_info['ID']=="10033-10082"]

In [16]:
#del events_with_line_info

In [ ]:
deps,drops=get_delay_causes()


In [27]:

events=[] 
for year in years_to_process:
    for line in [2,147]:
        #esemény és időjárás adat
        e = get_events_data(weather_df,coord_dict,deps,drops,year,line)
        e = add_line_info(e,line_status)
        e = add_previous_train_delay(e)
        e.fillna(0.0,inplace=True)
        events.append(e)
data=pd.concat(events)

C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\942977587.py:131: DtypeWarning: Columns (16,17,28) have mixed types. Specify dtype option on import or set low_memory=False.
  e=pd.read_csv(l+event_file_name+str(line)+'.txt',sep=',',encoding='iso-8859-2')
C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\942977587.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  events['ELOZO_ESEMENY_IDO'].fillna(events['IDO'],inplace=True)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\942977587.py:48: FutureWarning: A value is trying to

Before drop:  (3855893, 35)
After drop:  (2789964, 34)


C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\942977587.py:113: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  events=events.fillna(0.0)


2022-01-01 00:00:00  -  2023-12-31 00:00:00


C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\3499998205.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\3499998205.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\3499998205.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\

...

C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\3622305359.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e[['ELOZO_VONAT_KESES (m)','ELOZO_STATUS','ELOZO_VONAT_RELATIV_KESES']].fillna(0.0,inplace=True)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\942977587.py:131: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  e=pd.read_csv(l+event_file_name+str(line)+'.txt',sep=',',encoding='iso-8859-2')
C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\942977587.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a

Before drop:  (227466, 35)
After drop:  (106431, 34)


C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\942977587.py:113: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  events=events.fillna(0.0)


2023-01-01 00:00:00  -  2023-12-31 00:00:00


C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\3499998205.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[946 950 955 ... 494 505 505]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\3499998205.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Temp\ipykernel_19688\3499998205.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  e.update(joined)
C:\Users\ELTE\AppData\Local\Te

In [ ]:
# TODO nem az igazi, pkl helyette most
#save_data_with_types(data,data_root+'data.csv')

    

In [29]:
data.fillna(0.0,inplace=True)

In [30]:
data.to_pickle(data_root+'data.pkl')

In [28]:
data.head(30)

,VONATSZAM,KOZLEKEDESI_NAP,IDO,TENY_IDO,TERV_IDOTARTAM (m),KESES (m),OSSZ_KESES (m),TENY_IDOTARTAM (m),RELATIV_KESES,ELOZO_SZAKASZ_TERV_IDOTARTAM (m),...,ÁI,ÁE,Kezdés,Befejezés,"Kizárás (1=VZ,2=LJ)",Kiutalt menetvonalak zavartatása nélkül,Státus,ELOZO_VONAT_KESES (m),ELOZO_VONAT_RELATIV_KESES,VONAL_STATUSZ_VALTOZOTT
0,2010,2022-01-01,2022-01-01 00:25:00,2022-01-01 00:25:24,0.0,0.000000,0.400000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
1,2010,2022-01-01,2022-01-01 00:29:00,2022-01-01 00:30:10,4.0,0.766667,1.166667,4.766667,0.087452,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
2,2010,2022-01-01,2022-01-01 00:30:00,2022-01-01 00:30:40,1.0,-0.500000,0.666667,0.500000,-0.333333,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
3,2010,2022-01-01,2022-01-01 00:35:00,2022-01-01 00:34:55,5.0,-0.750000,-0.083333,4.250000,-0.081081,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
4,2010,2022-01-01,2022-01-01 00:36:00,2022-01-01 00:36:09,1.0,0.233333,0.150000,1.233333,0.104478,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
5,2010,2022-01-01,2022-01-01 00:38:00,2022-01-01 00:37:44,2.0,-0.416667,-0.266667,1.583333,-0.116279,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
6,2010,2022-01-01,2022-01-01 00:38:00,2022-01-01 00:38:17,0.0,0.550000,0.283333,0.550000,1.000000,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
7,2010,2022-01-01,2022-01-01 00:41:00,2022-01-01 00:40:58,3.0,-0.316667,-0.033333,2.683333,-0.055718,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
8,2010,2022-01-01,2022-01-01 00:42:00,2022-01-01 00:42:12,1.0,0.233333,0.200000,1.233333,0.104478,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
9,2010,2022-01-01,2022-01-01 00:43:00,2022-01-01 00:43:44,1.0,0.533333,0.733333,1.533333,0.210526,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True


# Késési ok meta

In [32]:
dc={}
reasonings=[]
for f in files:
    if not f.startswith('Vonat'):
        if f.startswith('Torzs'):
            delay_category_tables = pd.read_excel(delay_causes_dir+f,sheet_name=['Esemeny(EsemID)', 'KesesiFokod(KfkID)','KesesiSegedkod(KskID)'])
        print("Ignoring file: {}".format(f))
        continue
    l=f.replace('.xlsx','').split("_")[-1]
    if not l in dc.keys():
        dc[l]=[]
    delay_cause_tables = pd.read_excel(delay_causes_dir+f,sheet_name=['Vonat', 'VonatEsemeny','KesesIndoklas'])
    dc[l].append(delay_cause_tables)
for line,l in dc.items():
    for dfs in l:
        delay_causes=dfs['KesesIndoklas'].copy()
        delay_causes['Line']=line
        reasonings.append(delay_causes)
reasonings_df=pd.concat(reasonings, ignore_index=True)



Ignoring file: Torzs.xlsx
Ignoring file: ~$Torzs.xlsx
Ignoring file: ~$Vonat_2022_2023_147.xlsx


In [7]:
reasonings_df.head()

,VeID,VkOkSszam,KfkId,SkodId,IgPerc,Indoklas,Vonatszam,VonatDatum,RDEAzon,Line
0,188313858,1,22,36.0,1,Rákosrendező a local vg-on a 29+50-30sz. szel...,NaN,NaT,NaN,2
1,188314407,1,22,36.0,1,Rákosrendező a local vg-on a 29+50-30sz. szel...,NaN,NaT,NaN,2
2,188314939,1,22,36.0,1,Rákosrendező a local vg-on a 29+50-30sz. szel...,NaN,NaT,NaN,2
3,188316339,1,22,36.0,3,Rákosrendező a local vg-on a 29+50-30sz. szel...,NaN,NaT,NaN,2
4,188316340,1,13,29.0,1,utasforgalom,NaN,NaT,NaN,2


In [8]:
train_causes=reasonings_df[~reasonings_df['Vonatszam'].isnull()]
print(train_causes.shape)
explainable=train_causes[train_causes['Vonatszam'].isin(collected_trains[147]+collected_trains[2])].copy()
print(explainable.shape)
not_explainable=train_causes[~train_causes['Vonatszam'].isin(collected_trains[147]+collected_trains[2])].copy()
print(not_explainable.shape)



(68590, 10)
(52080, 10)
(16510, 10)


In [ ]:
train_causes.merge()

In [9]:
explainable.head()

,VeID,VkOkSszam,KfkId,SkodId,IgPerc,Indoklas,Vonatszam,VonatDatum,RDEAzon,Line
12,188331612,1,7,NaN,1,3265 +2',3265.0,2022-01-01,NaN,2
33,188365124,1,7,NaN,2,érk:+2,3277.0,2022-01-02,NaN,2
38,188382400,1,7,NaN,1,3283 +1',3283.0,2022-01-02,NaN,2
45,190293306,1,119,NaN,4,2066-ra várt,2066.0,2022-02-11,NaN,2
50,190294597,1,119,NaN,3,Vár 2066 érk.,2066.0,2022-02-11,NaN,2


In [10]:
reasonings_df.shape

(136969, 10)

In [ ]:
delay_category_tables['KesesiFokod(KfkID)'][['kfk_id','kfk_megnevezes']]

In [18]:
delay_categories=reasonings_df['KfkId'].value_counts()
delay_categories

KfkId
119    42318
110    29091
7      16851
22     16754
13     11153
6       8581
123     3439
118     3354
145     1550
2        756
142      712
135      411
133      310
129      282
188      255
136      168
156      125
171      124
19       121
134       96
154       87
151       74
155       58
172       51
126       45
153       39
138       35
150       34
169       27
140       20
173       13
161        6
175        6
130        5
127        4
181        4
174        2
148        2
122        2
191        2
132        1
103        1
Name: count, dtype: int64

In [19]:
delay_stat_df=pd.DataFrame(delay_categories).merge(delay_category_tables['KesesiFokod(KfkID)'][['kfk_id','kfk_megnevezes']],left_on='KfkId',right_on='kfk_id',how='left')

In [20]:
print(delay_stat_df.to_latex())


\begin{tabular}{lrrl}
\toprule
 & count & kfk_id & kfk_megnevezes \\
\midrule
0 & 42318 & 119 & Vonatforgalom szabályozás más vonat késése miatt \\
1 & 29091 & 110 & Menetidő túllépés \\
2 & 16851 & 7 & Fordulásra várás \\
3 & 16754 & 22 & Ideiglenes sebességkorlátozás \\
4 & 11153 & 13 & Tartózkodási idő túllépése \\
5 & 8581 & 6 & Csatlakozásra várás \\
6 & 3439 & 123 & Vasúttársasági előkészítés \\
7 & 3354 & 118 & Vonatforgalom szabályozás a vonat korábbi (saját) késése miatt \\
8 & 1550 & 145 & Vonatszemélyzeti okok \\
9 & 756 & 2 & Vonatforgalom szabályozás megrendelői igények alapján \\
10 & 712 & 142 & Forgalmi személyzeti okok  \\
11 & 411 & 135 & Látszólagos foglaltság \\
12 & 310 & 133 & Biztosítóberendezési zavar \\
13 & 282 & 129 & Vontatójármű zavarai \\
14 & 255 & 188 & Sebességkorlátozás külön rendelet alapján \\
15 & 168 & 136 & Útátjáró biztosítóberendezési zavar \\
16 & 125 & 156 & Idegen behatások \\
17 & 124 & 171 & Tervtől eltérő vonatösszeállítás vasútvállalati o

In [ ]:
Csinálunk egy dependency table-t ahol megjelenik, melyik vonatra vár az adott vonatunk, hol. A várt vonat aktuális(?) késését aztán az adott szakaszon (állomáson) bevesszük az attribútumok közé.

Ahol nincs információnk az adott vonatról a historikus adatokban, azokat a vonatokat eldobjuk.
